In [1]:
import syft as sy
import pytest
import numpy as np
from syft import Worker
from syft.service.user.user_roles import ServiceRole
from syft.service.service import ServiceConfigRegistry
from syft.node.credentials import SyftSigningKey
from syft.service.user.user_service import UserService, UserUpdate
from syft.service.context import AuthedServiceContext
from syft.client.api import SyftAPICall
from syft.client.client import SyftClient
from syft import SyftError
from syft import UID
from faker import Faker
from syft.store.mongo_document_store import StoreConfig
from syft.store.mongo_document_store import MongoStoreConfig
from syft.store.mongo_document_store import MongoStoreClientConfig
from syft.store.mongo_document_store import MongoStoreClientConfig
from syft.store.sqlite_document_store import SQLiteStoreClientConfig
from syft.store.sqlite_document_store import SQLiteStoreConfig
from syft.service.action.action_permissions import ActionObjectREAD
from syft.node.worker import Worker
from syft.node.node import create_worker_metadata
from syft import ActionObject
from syft.types.twin_object import TwinObject
from syft.service.action.action_object import TraceResult
from syft.service.action.plan import Plan, planify, build_plan_inputs
from syft.service.request.request import SubmitRequest, ActionStoreChange
from syft.store.linked_obj import LinkedObject
from syft.service.action.action_store import ActionPermission
from typing import List, Dict, Callable
from syft.service.action.action_object import Action
from collections.abc import Iterable
import inspect
import numpy as np

kj/filesystem-disk-unix.c++:1703: warning: PWD environment variable doesn't match current directory; pwd = /Users/koen/workspace/pysyft


In [2]:
# !rm /var/folders/q1/ryq93kwj055dlbpngxv1c7z40000gn/T/7bca415d13ed1ec841f0d0aede098dbb.sqlite

In [3]:
# worker = sy.Worker.named("test-domain-1", processes=1, reset=True)
worker = sy.Worker.named("eager_exec", reset=True, processes=0)
client = worker.root_client
guest_client = worker.guest_client

# Simple permissions

In [4]:
input_obj = TwinObject(
    private_obj=np.array([[3,3,3], [3,3,3]]),
    mock_obj=np.array([[1,1,1], [1,1,1]])
)

In [5]:
input_ptr = client.api.services.action.set(input_obj)

pointer = guest_client.api.services.action.get_pointer(input_ptr.id) 

In [6]:
pointer.id

<UID: acd67438c02d412892dd8fdd7db1da63>

In [7]:
store = worker.get_service("actionservice").store

In [8]:
input_ptr = client.api.services.action.set(input_obj)

pointer = guest_client.api.services.action.get_pointer(input_ptr.id) 

flat_ptr = pointer.flatten()

res_guest = guest_client.api.services.action.get(flat_ptr.id)
assert not isinstance(res_guest, ActionObject)
res_root = flat_ptr.get_from(client)
assert all(res_root == [3,3,3,3,3,3])

# Plans

In [9]:
# worker = sy.Worker.named("test-domain-1", processes=1, reset=True)
worker = sy.Worker.named("eager_exec", reset=True, processes=0)
client = worker.root_client
guest_client = worker.guest_client

## Plans with object instantiations

In [10]:
@planify
def my_plan(x=np.array([1,2,3,4,5,6])):
    return x+1

In [11]:
plan_ptr = my_plan.send(guest_client)

In [12]:
input_obj = TwinObject(
    private_obj=np.array([1,2,3,4,5,6]),
    mock_obj=np.array([1,1,1,1,1,1])
)


In [13]:
_id = client.api.services.action.set(input_obj).id
pointer = guest_client.api.services.action.get_pointer(_id) 

In [14]:
res_ptr = plan_ptr(x=pointer)

In [15]:
res_ptr.get_from(client)

array([2, 3, 4, 5, 6, 7])

## Complex plans

In [16]:
@planify
def my_plan(x=np.array([1,2,3,4,5,6])):
    y = x.flatten() # method -> [1,2,3,4,5,6]
    min_val = x.min() # method -> 1
    s = x.shape # getattribute -> 6    
    w = x[min_val] # __getitem__ -> 2
    y[min_val] = min_val # __setitem__ y = [1,1,3,4,5,6]
    res = client.api.lib.numpy.sum(y) + w + s # client function and __add__ 20 + 2 + 6 = 28
    return res #28

In [17]:
my_plan

```python
Pointer
```
```python
class Plan:
  id: str = c197b90d939e4703899c08101473c9a4
  inputs: str = {'x': Pointer:
[1 2 3 4 5 6]}
  outputs: str = [Pointer:
[28]]
  actions: str = [ActionObject NumpyArrayObject[182..b].flatten(,), ActionObject NumpyArrayObject[182..b].min(,), ActionObject NumpyArrayObject[182..b].shape(,), ActionObject NumpyArrayObject[182..b].__getitem__(f29..8,), ActionObject NumpyArrayObject[1d8..b].__setitem__(f29..8, f29..8,), ActionObject numpy.sum(1d8..b,), ActionObject NumpyScalarObject[058..2].__add__(550..6,), ActionObject NumpyScalarObject[850..f].__add__(555..c,)]
  code: str = "@planify
def my_plan(x=np.array([1,2,3,4,5,6])):
    y = x.flatten() # method -> [1,2,3,4,5,6]
    min_val = x.min() # method -> 1
    s = x.shape # getattribute -> 6    
    w = x[min_val] # __getitem__ -> 2
    y[min_val] = min_val # __setitem__ y = [1,1,3,4,5,6]
    res = client.api.lib.numpy.sum(y) + w + s # client function and __add__ 20 + 2 + 6 = 28
    return res #28
"
  client: str = None
  syft_passthrough_attrs: str = ['inputs', 'outputs', 'code', 'actions', 'client']

```

In [18]:
plan_ptr = my_plan.send(guest_client)

In [19]:
input_obj = TwinObject(
    private_obj=np.array([1,2,3,4,5,6]),
    mock_obj=np.array([1,1,1, 1,1,1])
)

_id = client.api.services.action.set(input_obj).id
pointer = guest_client.api.services.action.get_pointer(_id) 

In [20]:
res_ptr = plan_ptr(x=pointer)

In [21]:
res_ptr.get_from(client)

array([28])

## Plans with function calls

In [22]:
store = worker.get_service("actionservice").store

In [23]:
client.api.lib.numpy.sum

<function syft.client.api.generate_remote_lib_function.<locals>.wrapper(*args, **kwargs)>

In [24]:
@planify
def my_plan(x=np.array([[2,2,2], [2,2,2]])):
    y = x.flatten()
    w = client.api.lib.numpy.sum(y)
    return w

In [25]:
plan_ptr = my_plan.send(guest_client)

input_obj = TwinObject(
    private_obj=np.array([[3,3,3], [3,3,3]]),
    mock_obj=np.array([[1,1,1], [1,1,1]])
)

input_ptr = client.api.services.action.set(input_obj)
pointer = guest_client.api.services.action.get_pointer(input_ptr.id) 


In [28]:
res_ptr = plan_ptr(x=pointer)

In [29]:
assert res_ptr.get_from(client) == 18

## simple execution (root downloads)

In [30]:
store = worker.get_service("actionservice").store

In [31]:
@planify
def my_plan(x=np.array([[2,2,2], [2,2,2]])):
    y = x.flatten()
    z = y.prod()
    return z


In [32]:
plan_ptr = my_plan.send(guest_client)

In [33]:
input_obj = TwinObject(
    private_obj=np.array([[3,3,3], [3,3,3]]),
    mock_obj=np.array([[1,1,1], [1,1,1]])
)

In [34]:
input_ptr = client.api.services.action.set(input_obj)


In [35]:
pointer = guest_client.api.services.action.get_pointer(input_ptr.id) 

In [36]:
res_ptr = plan_ptr(x=pointer)

In [37]:
store.data[input_ptr.id]

```python
class TwinObject:
  id: str = 577f7374edf6496694038193381f4d96
  private_obj: str = [[3 3 3]
 [3 3 3]]
  private_obj_id: str = 6cca3e484ded49b0b9fcb886783161fb
  mock_obj: str = [[1 1 1]
 [1 1 1]]
  mock_obj_id: str = f273dc6bb99741b6bff7356ea1f99947

```

In [38]:
# todo: should be 1
res_ptr

```python
Pointer
```
64

In [40]:
assert not isinstance(guest_client.api.services.action.get(res_ptr.id), ActionObject)

In [41]:
assert res_ptr.get_from(client) == np.array([[3,3,3], [3,3,3]]).flatten().prod()

## Downloading the result

In [42]:
res_ptr.request(guest_client)

```python
class Request:
  id: str = 1b915e2d8f0b450a9434d7f577ebd65a
  requesting_user_verify_key: str = b4068829a465455c75c90a584acff9b9e7e5bcf7d5b21290d21ab30a8df0861f
  approving_user_verify_key: str = None
  request_time: str = 2023-05-09 14:10:52
  approval_time: str = None
  status: str = RequestStatus.PENDING
  node_uid: str = 20d3b7820d29e7ffdc7c53435fcad1bd
  request_hash: str = "786c74e396ae64850543b520ba0ac089032cdffef9b6af2aa952773c896c4b41"
  changes: str = [syft.service.request.request.ActionStoreChange]

```

In [43]:
client.api.services.request[0].approve_with_client(client)

<class 'syft.service.response.SyftSuccess'>: Request 1b915e2d8f0b450a9434d7f577ebd65a changes applied

In [44]:
res_ptr.get_from(guest_client)

729

# Pointers (Twins)

## setattribute

In [45]:
worker = sy.Worker.named("eager_exec", reset=True, processes=0)
client = worker.root_client
guest_client = worker.guest_client

In [46]:
private_data, mock_data = np.array([[1.0,2.0,3.0], [4.0,5.0,6.0]]), np.array([[1.0,1.0,1.0], [1.0,1.0,1.0]])

In [47]:
obj = TwinObject(
    private_obj=private_data,
    mock_obj=mock_data
)

In [48]:
obj_pointer = client.api.services.action.set(obj)
obj_pointer = guest_client.api.services.action.get_pointer(obj_pointer.id)

In [49]:
original_id = obj_pointer.id

In [50]:
obj_pointer

```python
TwinPointer(Mock)
```
array([[1., 1., 1.],
       [1., 1., 1.]])

In [56]:
# note that this messes up the data and the shape of the array
obj_pointer.dtype = np.int32  

In [57]:
# local object is updated
assert obj_pointer.id.id in worker.action_store.data
assert obj_pointer.id != original_id

In [58]:
res = obj_pointer.get_from(client)

In [59]:
assert res.dtype == np.int32

In [60]:
private_data.dtype= np.int32
mock_data.dtype= np.int32

In [61]:
assert (res == private_data).all()
assert (obj_pointer.syft_action_data == mock_data).all()
assert not (obj_pointer.syft_action_data == private_data).all()

## getattribute

In [62]:
worker = sy.Worker.named("eager_exec", reset=True, processes=0)
client = worker.root_client
guest_client = worker.guest_client

In [63]:
obj = TwinObject(
    private_obj=np.array([[1,2,3], [4,5,6]]),
    mock_obj=np.array([[1,1,1], [1,1,1]])
)

In [64]:
obj_pointer = client.api.services.action.set(obj)
obj_pointer = guest_client.api.services.action.get_pointer(obj_pointer.id)

In [65]:
size_pointer = obj_pointer.size   

In [66]:
assert size_pointer.id.id in worker.action_store.data

In [67]:
assert size_pointer.get_from(client) == 6

## methods

In [68]:
worker = sy.Worker.named("eager_exec", reset=True, processes=0)
client = worker.root_client
guest_client = worker.guest_client

In [69]:
obj = TwinObject(
    private_obj=np.array([[1,2,3], [4,5,6]]),
    mock_obj=np.array([[1,1,1], [1,1,1]])
)

In [70]:
obj_pointer = client.api.services.action.set(obj)
obj_pointer = guest_client.api.services.action.get_pointer(obj_pointer.id)

In [71]:
flat_pointer = obj_pointer.flatten()   

In [72]:
assert flat_pointer.id.id in worker.action_store.data

In [73]:
assert all(flat_pointer.get_from(client) == np.array([1,2,3,4,5,6]))

## Indexing

In [74]:
worker = sy.Worker.named("eager_exec", reset=True, processes=0)
client = worker.root_client
guest_client = worker.guest_client

In [75]:
obj = TwinObject(
    private_obj=np.array([[1,2,3], [4,5,6]]),
    mock_obj=np.array([[1,1,1], [1,1,1]])
)

In [76]:
obj_pointer = client.api.services.action.set(obj)
obj_pointer = guest_client.api.services.action.get_pointer(obj_pointer.id)

In [78]:
indexed_pointer = obj_pointer[0, 0:2]   

In [79]:
assert indexed_pointer.id.id in worker.action_store.data

In [80]:
assert all(indexed_pointer.get_from(client) == np.array([1,2]))